# FLearner

> Fill in a module description here


In [ ]:
#| default_exp FLearner

In [ ]:
#| hide
from nbdev.showdoc import *  # noqa: F403

## ToDo

- Add the TextBlock and The TabularBlock.
- Add loging using Loguru.

In [ ]:
#| export
import torch
import os
from datetime import datetime
from fastcore.utils import * # type: ignore # noqa: F403
from fedai.federated.agents import * # noqa: F403
from fedai.learner_utils import * # type: ignore # noqa: F403
from fedai.client_selector import *  # noqa: F403
from fedai.core import get_cfg  # noqa: F401, F403
from fedai.wandb_writer import *  # noqa: F403
from torch import nn
from omegaconf import OmegaConf
import argparse
import yaml

In [ ]:
#| export
def client_fn(client_cls, cfg, id, latest_round, t, loss_fn = None, optimizer = None):
    
    model = get_model(cfg)
    criterion = get_criterion(loss_fn)

    train_block = get_block(cfg, id)
    test_block = get_block(cfg, id, train=False)    
    
    state = {'model': model, 'optimizer': None, 'criterion': criterion, 't': t}
    
    if t > 1:
        state = load_state_from_disk(cfg, state, latest_round, id, t)  # noqa: F405
    
    state['optimizer'] = get_cls("torch.optim", cfg.optimizer.name)(state['model'].parameters(), lr=cfg.lr)
    
    return client_cls(id, cfg, state, block= [train_block, test_block])


In [ ]:
#| export
class FLearner:
    def __init__(self,
                 cfg, # OmegaConf object
                 client_fn, # a function that returns a client object
                 client_selector= BaseClientSelector, # a client selection class represnting a client seleection algorithm # noqa: F405
                 client_cls= FLAgent,  # noqa: F405
                 loss_fn= torch.nn.CrossEntropyLoss,  # noqa: F405
                 writer= WandbWriter): # a writer to write results to an expirement tracking tool # noqa: F405
        
        self.cfg = cfg
        self.cfg.now = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.cfg.save_dir = os.path.join(self.cfg.root_dir, self.cfg.now, self.cfg.save_dir)
        self.cfg.log_dir = os.path.join(self.cfg.root_dir, self.cfg.now, self.cfg.log_dir)
        self.cfg.res_dir = os.path.join(self.cfg.root_dir, self.cfg.now, self.cfg.res_dir)

        self.client_fn = client_fn
        
        self.client_selector = client_selector(self.cfg)
        self.client_cls = client_cls
        self.loss_fn = loss_fn()
        self.writer = writer(cfg)
        self.server  = self.client_cls(cfg= self.cfg, block= None, id= -1, state= None, role= AgentRole.SERVER)  # noqa: F405
        self.server.server_init(self.client_fn, self.client_selector, self.client_cls, self.loss_fn, self.writer)
    

In [ ]:
#| export
@patch
def run_simulation(self: FLearner):
    self.server.runFL()

In [ ]:
# #| export
# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description='Run Federated Learning Simulation')
#     parser.add_argument('--config', type=str, help='Path to the config file', required=True)
#     args = parser.parse_args()

#     try:
#         with open(args.config, 'r') as file:
#             cfg = yaml.safe_load(file)
#             cfg = OmegaConf.create(cfg)
#     except:
#         print("Invalid config file path")
    
#     client_selector = get_cls("fedai.client_selector", cfg.client_selector)
#     client_cls = get_cls("fedai.federated.agents", cfg.client_cls)
#     loss_fn = get_cls("torch.nn", cfg.loss_fn)
#     writer = get_cls("fedai.wandb_writer", cfg.writer)


#     learner = FLearner(cfg, client_fn, client_selector, client_cls, loss_fn, writer)
#     learner.run_simulation()

In [ ]:
# #| export
# @patch
# def run_simulation(self: FLearner):
#     res =  []
#     all_ids = self.client_selector.select()
    
#     for t in range(1, self.cfg.n_rounds):
#         lst_active_ids = all_ids[t]
#         len_clients_ds = []
#         round_res = []

#         for id in lst_active_ids:
#             client = self.client_fn(self.client_cls, self.cfg, id, self.latest_round, t, self.loss_fn)
#             len_clients_ds.append(len(client.train_ds))
            
#             self.server.communicate(client) 

#             client_history = client.fit() 
#             round_res.append(client_history)
#             res.append(round_res)

#             client.communicate(self.server) 
#             self.latest_round[id] = t 

#         one_model = True if self.server.cfg.agg == 'one_model' else False
#         self.server.aggregate(lst_active_ids, t, len_clients_ds, one_model= one_model) 
#         self.writer.write(round_res, t) 
        
#     self.writer.save(res)
#     self.writer.finish()

#     return res

In [ ]:
#| hide
import nbdev # type: ignore
nbdev.nbdev_export() # type: ignore  # noqa: E702
